In [33]:
maxN = 310
a = [[0] * maxN for _ in range(maxN)]  # 初始化一个二维数组a，用于存储输入的数
ans = [0] * maxN  # 初始化数组ans，存储计算结果

def main():
    n = int(input())  # 输入n，表示矩阵的大小
    for i in range(1, n + 1):
        values = list(map(int, input().split()))  # 输入第i行的数，并转换为整数列表
        for j, value in enumerate(values, start=i):  # 从第i列开始填入数据
            a[i][j] = value
    
    ans[1] = 0  # 初始化第一个结果为0
    ans[2] = ans[1] + a[1][2]  # 计算第二个结果
    last = 2  # 初始化last为2，表示上一个已计算的索引

    # 从第3个元素开始遍历到第n个元素
    for i in range(3, n + 1):
        if a[last-1][i] == a[last-1][last] + a[last][i]:  # 检查是否满足某个条件
            if ans[last] > ans[last-1]:
                ans[i] = ans[last] + a[last][i]  # 根据条件更新ans[i]
            else:
                ans[i] = ans[last] - a[last][i]
        elif a[last-1][i] < a[last-1][last] + a[last][i]:  # 检查另一个条件
            if ans[last] > ans[last-1]:
                ans[i] = ans[last] - a[last][i]
            else:
                ans[i] = ans[last] + a[last][i]
        if a[i-1][i] != 0:  # 如果a[i-1][i]不是0，更新last
            last = i

    for i in range(1, n + 1):  # 输出结果数组
        print(ans[i], end=" ")
    print()

if __name__ == "__main__":
    main()

 5
 0 1 3 3 3
 0 2 2 2
 0 1 2
 0 1
 0


0 1 3 2 1 


In [45]:
def char_to_index(char):
    return ord(char) - ord('A') + 1 if char.isupper() else ord(char) - ord('a') + 27

def main():
    test_cases = int(input())  # 输入测试数据的数量
    for _ in range(test_cases):
        mapping = [0] * 53  # 存储字符映射关系
        indegree = [0] * 53  # 存储每个字符的入度
        start_group = [0] * 53  # 用于环检测

        source_string = input().strip()  # 输入源字符串去空格
        target_string = input().strip()  # 输入目标字符串

        target_set = set()  # 存储目标字符的集合
        is_convertible = True  # 标记是否可转换
        is_same = True  # 标记是否所有字符相同
        length = len(source_string)  # 字符串长度

        for i in range(length):
            source_index = char_to_index(source_string[i])
            target_index = char_to_index(target_string[i])
            if mapping[source_index] and mapping[source_index] != target_index:
                is_convertible = False  # 同一源字符指向不同目标字符
                break
            mapping[source_index] = target_index  # 更新映射关系
            target_set.add(target_index)  # 添加目标字符到集合
            if source_string[i] != target_string[i]:
                is_same = False  # 标记字符不同

        if len(target_set) == 52 and not is_same:
            is_convertible = False  # 所有字符都不同且不可转换

        if not is_convertible:
            print("-1")
            continue

        if is_same:
            print("0")
            continue

        operation_count = 0  # 记录需要的操作数
        for i in range(1, 53):
            if mapping[i] and mapping[i] != i:
                operation_count += 1
                indegree[mapping[i]] += 1

        for i in range(1, 53):
            current = i
            if start_group[current]:
                continue
            while current and not start_group[current]:
                start_group[current] = i  # 标记同一环
                current = mapping[current]
            if current and current != mapping[current] and start_group[current] == i:
                has_multiple_indegree = False  # 标记是否有多重入度
                cycle_node = current
                while True:
                    if indegree[cycle_node] > 1:
                        has_multiple_indegree = True
                    cycle_node = mapping[cycle_node]
                    if cycle_node == current:
                        break
                if not has_multiple_indegree:
                    operation_count += 1  # 增加操作数

        print(operation_count)

if __name__ == "__main__":
    main()

 1
 abc
 ccb


3


In [49]:
class Edge:
    def __init__(self, to_node=0, next_node=0):
        self.to_node = to_node
        self.next_node = next_node

class Answer:
    def __init__(self, from_node=0, to_node=0, w=0):
        self.from_node = from_node
        self.to_node = to_node
        self.w = w

def add_edge(edges, head, u, v, eID):
    edges[eID].to_node = v
    edges[eID].next_node = head[u]
    head[u] = eID

def dfs1(u, fa, h, edges, head, ans):
    eID = head[u]
    while eID:
        v = edges[eID].to_node
        if v != fa:
            dfs1(v, u, h, edges, head, ans)
            if h[v] > 0:
                ans.append(Answer(v, u, h[v]))
            h[u] += h[v]
        eID = edges[eID].next_node

def dfs2(u, fa, h, edges, head, ans):
    eID = head[u]
    while eID:
        v = edges[eID].to_node
        if v != fa:
            if h[v] < 0:
                ans.append(Answer(u, v, -h[v]))
            dfs2(v, u, h, edges, head, ans)
        eID = edges[eID].next_node

def main():
    n = int(input().strip())
    hay_counts = list(map(int, input().strip().split()))
    
    h = [0] * (n + 1)
    sum_h = sum(hay_counts)
    
    for i in range(1, n + 1):
        h[i] = hay_counts[i - 1] - sum_h // n

    edges = [Edge() for _ in range(2 * n)]
    head = [0] * (n + 1)
    eID = 0
    ans = []

    for _ in range(n - 1):
        u, v = map(int, input().strip().split())
        eID += 1
        add_edge(edges, head, u, v, eID)
        eID += 1
        add_edge(edges, head, v, u, eID)

    dfs1(1, 0, h, edges, head, ans)
    dfs2(1, 0, h, edges, head, ans)

    print(len(ans))
    for a in ans:
        print(a.from_node, a.to_node, a.w)

if __name__ == "__main__":
    main()

 6
 9 7 8 2 2 2
 1 2
 2 4
 1 3
 3 5
 3 5


5
1 3 3
3 5 3
3 5 3
1 2 1
2 4 3


In [39]:
source_string = input().strip()
print(source_string)

    32 23 2


32 23 2


In [51]:
# 接收用户输入的成绩
grade = float(input("请输入学生的成绩："))

# 根据成绩判断等级
if grade >= 90:
    print("A")
elif grade >= 80:
    print("B")
elif grade >= 60:
    print("C")
else:
    print("D")


请输入学生的成绩： 83.6


B


In [59]:
a=int(input("pls input a 3 digits number"))
b=a%10
c=a//10%10
d=a//100
print(b)
print(c)
print(d)

pls input a 3 digits number 145


5
4
1


In [66]:
a=int(input())
b=int(input())
c=int(input())
if a>c and a>b:
    max=a
elif b>c and b>c :
    max=b
else:
    max=c
print("max=", max)
    

 5
 6
 4


max= 6
